In [1]:
import pandas as pd
from datetime import datetime, timedelta

year = pd.Timestamp('2025-01-01 12:00:00')


In [12]:
import json
import pandas as pd
from glob import glob
import os

folder_path = "./Documents/spotifyData"
files = glob(os.path.join(folder_path, "*.json"))

dfs = []
for file in files:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        dfs.append(pd.DataFrame(data))
df = pd.concat(dfs, ignore_index=True)
print(df.head())

                     ts platform  ms_played conn_country  \
0  2024-06-30T00:25:47Z      ios       2720           US   
1  2024-06-30T00:28:38Z      ios     171818           US   
2  2024-06-30T00:33:00Z      ios     262000           US   
3  2024-06-30T00:34:37Z      ios      96000           US   
4  2024-06-30T00:37:54Z      ios     197142           US   

                                  ip_addr  \
0  2600:100f:b1a5:9100:50b9:ba9d:188:8f82   
1  2600:100f:b1a5:9100:50b9:ba9d:188:8f82   
2  2600:100f:b1a5:9100:50b9:ba9d:188:8f82   
3  2600:100f:b1a5:9100:50b9:ba9d:188:8f82   
4  2600:100f:b1a5:9100:50b9:ba9d:188:8f82   

            master_metadata_track_name master_metadata_album_artist_name  \
0  Y U DON'T LOVE ME? (MISS AMERIKKKA)                       Joey Bada$$   
1                              D'Evils                               SiR   
2                                 Leak                   mynameisntjmack   
3                    AIN'T GON STOP ME                          